In [ ]:
import time
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
import BranchedGP

In [ ]:
import sys
import os

In [ ]:
plt.style.use("ggplot")
%matplotlib inline


In [ ]:
Y = pd.read_csv("../results/scRNAseq_R13547_10x_mNT_20220813/RA_symetryBreaking/branching_genes_BGP/symmetry_breaking_scRNAseq_data_tfs.csv", index_col=[0])
monocle = pd.read_csv("../results/scRNAseq_R13547_10x_mNT_20220813/RA_symetryBreaking/branching_genes_BGP/symmetry_breaking_dpt.csv", index_col=[0])

In [ ]:
outDir = "../results/scRNAseq_R13547_10x_mNT_20220813/RA_symetryBreaking/branching_genes_BGP/BGP_out_tfs/"

In [ ]:
Y.head()

In [ ]:
monocle.head()

In [ ]:
Y.shape

In [ ]:
# Plot Monocle DDRTree space
genelist = ["Foxa2", "Pax6", "Skil"]
f, ax = plt.subplots(1, len(genelist), figsize=(10, 5), sharex=True, sharey=True)
for ig, g in enumerate(genelist):
    y = Y[g].values
    yt = np.log(1 + y / y.max())
    yt = yt / yt.max()
    h = ax[ig].scatter(
        monocle["DC_1"],
        monocle["DC_4"],
        c=yt,
        s=50,
        alpha=1.0,
        vmin=0,
        vmax=1,
    )
    ax[ig].set_title(g)

In [ ]:
def PlotGene(label, X, Y, s=3, alpha=1.0, ax=None):
    fig = None
    if ax is None:
        fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    for li in np.unique(label):
        idxN = (label == li).flatten()
        ax.scatter(X[idxN], Y[idxN], s=s, alpha=alpha, label=int(np.round(li)))
    return fig, ax


In [ ]:
# np.linspace(0.05, 0.95, 5)

In [ ]:
def FitGene(g, ns=20):  # for quick results subsample data
    t = time.time()
    Bsearch = list(np.linspace(0.05, 0.95, 5)) + [
        1.1
    ]  # set of candidate branching points
    GPy = (Y[g].iloc[::ns].values - Y[g].iloc[::ns].values.mean())[
        :, None
    ]  # remove mean from gene expression data
    GPt = monocle["pseudotime"].values[::ns]
    globalBranching = monocle["state"].values[::ns].astype(int)
    d = BranchedGP.FitBranchingModel.FitModel(Bsearch, GPt, GPy, globalBranching)
    print(g, "BGP inference completed in %.1f seconds." % (time.time() - t))
    # plot BGP
    fig, ax = BranchedGP.VBHelperFunctions.PlotBGPFit(
        GPy, GPt, Bsearch, d, figsize=(10, 10)
    )
    # overplot data
    f, a = PlotGene(
        monocle["state"].values,
        monocle["pseudotime"].values,
        Y[g].values - Y[g].iloc[::ns].values.mean(),
        ax=ax[0],
        s=10,
        alpha=0.5,
    )
    # Calculate Bayes factor of branching vs non-branching
    bf = BranchedGP.VBHelperFunctions.CalculateBranchingEvidence(d)["logBayesFactor"]

    fig.suptitle("%s log Bayes factor of branching %.1f" % (g, bf))
    return d, fig, ax, bf



In [ ]:
#gene = 'Foxa2'
#n = 0;
#outDir = "../results/scRNAseq_R13547_10x_mNT_20220813/RA_symetryBreaking/branching_genes_BGP/BGP_out/"
#d_c, fig, ax, bf = FitGene(gene);   
# save the plot
#fig.savefig(os.path.join(outDir, gene + "_branching_time.pdf"));     


In [ ]:
#d_c

In [ ]:
#bf

In [ ]:
#with open(os.path.join(outDir, gene + "_branching_time.txt"), 'w') as f:
#        f.write(gene + '\t' + str(braching_time) + '\t' + str(bf))   

In [ ]:
n = 0;
for gene in Y.columns:
    print(gene)
    n = n + 1 
    
    # gene = 'Foxa2'
    d_c, fig, ax, bf = FitGene(gene);
    
    # save the plot
    fig.savefig(os.path.join(outDir, gene + "_branching_time.pdf")); 
    
    braching_time = d_c['posteriorB']['Bmode']
    
    with open(os.path.join(outDir, gene + "_branching_time.txt"), 'w') as f:
        f.write(gene + '\t' + str(braching_time) + '\t' + str(bf))
    
    #if(n > 3):
    #    break
    

In [ ]:
fig
gene = 'Foxa2'

In [ ]:
outDir = "../results/scRNAseq_R13547_10x_mNT_20220813/RA_symetryBreaking/branching_genes_BGP/BGP_out/"
fig.savefig(os.path.join(outDir, gene+ "_branching_time.pdf")) 

In [ ]:
braching_time = d_c['posteriorB']['Bmode']
    
with open(os.path.join(outDir, gene + "_branching_time.txt"), 'w') as f:
    f.write(str(braching_time))

In [ ]:
braching_time = d_c['posteriorB']['Bmode']
braching_time

In [ ]:
d, fig, ax = FitGene("Foxa2")

In [ ]:
d_c, fig_c, ax_c = FitGene("Pax6")

In [ ]:
d_c, fig_c, ax_c = FitGene("Tcf15")

In [ ]:
d_c, fig_c, ax_c = FitGene("Peg3")

In [ ]:
d_c['posteriorB']


In [ ]:
d_c['posteriorB']['Bmode']

In [ ]:
np.linspace(0.05, 0.95, 10)

In [ ]:
d_c, fig_c, ax_c = FitGene("Lef1")

In [ ]:
import pickle

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from BranchedGP import VBHelperFunctions as bplot

plt.style.use("ggplot")
%matplotlib inline


In [ ]:
datafile = "analysis_examples/BranchedGP/notebooks/syntheticdata/synthetic20.csv"
data = pd.read_csv(datafile, index_col=[0])
G = data.shape[1] - 2  # all data - time columns - state column
Y = data.iloc[:, 2:]
trueBranchingTimes = np.array([float(Y.columns[i][-3:]) for i in range(G)])

In [ ]:
data.head()


In [ ]:
data.shape

In [ ]:
f, ax = plt.subplots(5, 8, figsize=(10, 8))
ax = ax.flatten()
for i in range(G):
    for s in np.unique(data["MonocleState"]):
        idxs = s == data["MonocleState"].values
        ax[i].scatter(data["Time"].loc[idxs], Y.iloc[:, i].loc[idxs])
        ax[i].set_title(Y.columns[i])
        ax[i].set_yticklabels([])
        ax[i].set_xticklabels([])
f.suptitle("Branching genes, location=1.1 indicates no branching")

In [ ]:
#Run the BGP model
# Run script runsyntheticData.py to obtain a pickle file with results. 
# This script can take ~10 to 20 minutes depending on your hardware. 
# It performs a gene-by-gene branch model fitting.


In [ ]:
r = pickle.load(open("analysis_examples/BranchedGP/notebooks/syntheticdata/syntheticDataRun.p", "rb"))

In [ ]:
r.keys()

In [ ]:
# plot fit for a gene
g = 0
GPy = Y.iloc[:, g][:, None]
GPt = data["Time"].values
globalBranching = data["MonocleState"].values.astype(int)
bmode = r["Bsearch"][np.argmax(r["gpmodels"][g]["loglik"])]
print("True branching time", trueBranchingTimes[g], "BGP Maximum at b=%.2f" % bmode)
_ = bplot.PlotBGPFit(GPy, GPt, r["Bsearch"], r["gpmodels"][g])

In [ ]:
g = 0
bmode = r["Bsearch"][np.argmax(r["gpmodels"][g]["loglik"])]
pred = r["gpmodels"][g]["prediction"]  # prediction object from GP


In [ ]:
pred["mu"];

In [ ]:
_ = bplot.plotBranchModel(
    bmode,
    GPt,
    GPy,
    pred["xtest"],
    pred["mu"],
    pred["var"],
    r["gpmodels"][g]["Phi"],
    fPlotPhi=True,
    fColorBar=True,
    fPlotVar=True,
)

In [ ]:
fs, ax = plt.subplots(1, 1, figsize=(5, 5))
for g in range(G):
    bmode = r["Bsearch"][np.argmax(r["gpmodels"][g]["loglik"])]
    ax.scatter(bmode, g, s=100, color="b")  # BGP mode
    ax.scatter(trueBranchingTimes[g] + 0.05, g, s=100, color="k")  # True